# RAG-U

RAG-based personalization model construction


In [25]:
import operator
import os
from typing import Annotated, Sequence, TypedDict

import chromadb
from dotenv import load_dotenv
from langchain_community.chat_models.litellm import ChatLiteLLM
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph

load_dotenv()

DB_DIR = os.getenv("DB_DIR", "./.db")

chroma_client = chromadb.PersistentClient(path=DB_DIR)
collection = chroma_client.get_or_create_collection("rag-u")

llm = ChatLiteLLM(model="groq/Llama3-70b-8192")

In [39]:
def call_oracle(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [48]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
USER MODEL:
{user_model}

Given the conversation so far and the user model, update the user model with anything new you've learned about the user.
         """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm


def call_learner(state):
    messages = state["messages"]
    user_model = state["user_model"]
    response = chain.invoke({"user_model": user_model, "messages": messages})
    return {"user_model": [response]}

In [49]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    user_model: str


graph = StateGraph(AgentState)

graph.add_node("oracle", call_oracle)
graph.add_node("learner", call_learner)
graph.add_edge("oracle", "learner")
graph.add_edge("learner", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [50]:
inputs = {"messages": [HumanMessage(content="I need you to help solve a problem")]}
runnable.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content="The weather in San Francisco!\n\nSan Francisco's climate is known for being cool and foggy, with a Mediterranean climate characterized by cool, wet winters and mild, dry summers. Here's a brief overview of what you can expect:\n\n**Current Weather:**\nTo get the current weather conditions in San Francisco, I'd recommend checking a reliable weather website or app, such as:\n\n* National Weather Service (NWS): [www.weather.gov](http://www.weather.gov)\n* Dark Sky (iOS, Android): A popular app that provides hyperlocal weather forecasts.\n* Weather Underground (web, iOS, Android): A crowdsourced weather service that provides detailed forecasts.\n\n**Typical Weather Patterns:**\n\n* **Summer (June to August):** Mild temperatures, with average highs around 67°F (19°C) and lows around 54°F (12°C). Expect some fog in the mornings, which usually burns off by mid-morning.\n* **Winter (December to February):** C

In [ ]:
demo = """
"Starfish: Demonstrating the Future of AI-Powered Professional Networking and Task Management"
In the forthcoming tech demo for the Starfish project, we aim to showcase the innovative use of Digital Doubles within our Work Marketplace, illustrating the seamless integration of AI-powered agents in enhancing professional workflows and facilitating project completions. The demonstration will focus on how Digital Doubles can effectively represent individuals, manage multi-agent collaborations, and utilize feedback mechanisms for continuous learning and personalization. Specifically, the demo will feature several scenarios where Digital Doubles orchestrate diverse AI tools and systems to accomplish designated tasks that are part of a larger project structure, based on the detailed skills profiles of their human counterparts.
This demonstration will simulate a realistic work environment where multiple professionals, equipped with their Digital Doubles, collaborate on a complex project. Here’s the envisioned flow:
Simulation of Skill Matching and Task Allocation: Display how Digital Doubles analyze available projects and tasks within the marketplace, aligning their human counterparts' skills and career aspirations with the needs of potential employers or project leaders.
Collaborative Project Execution: Show multiple Digital Doubles interacting with each other and with various specialized AI systems to perform tasks. This will demonstrate their capability to manage resources, time, and communication effectively, increasing project efficiency.
Dynamic Learning and Adaptation: Highlight how these Digital Doubles receive and integrate feedback in real-time to improve their performance and adapt their strategies to meet project goals better.
Outcome Showcase and Feedback Loop: At the end of the demo, the results of the project tasks managed by the Digital Doubles will be evaluated to demonstrate the efficacy of the digital representation and the multilayered AI interaction.
User Interaction and Control Over Digital Double: Briefly demonstrate how individuals can set preferences, supervise task execution, and refine the learning parameters of their Digital Doubles, ensuring they remain aligned with personal growth and professional objectives.
Technological Integration and Data Security: Explain the underlying technology that enables seamless orchestration of AI agents, the personalization processes for individual learning, and robust data security measures that protect users' information and uphold trust in the digital platform.
This tech demo aims not only to illustrate the capabilities of Digital Doubles but also to foster discussions on the potential directions and expansions possible within the LinkedIn ecosystem, empowering users to harness the full potential of AI in the evolving digital economy. This will help us refine our approach, gather valuable feedback, and drive continued innovation in the application of AI in professional networking and task management.
"""

In [1]:
import os
from operator import itemgetter

from dotenv import load_dotenv
from langchain_community.chat_models.litellm import ChatLiteLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

#

load_dotenv()

DB_DIR = os.getenv("DB_DIR", "./.db")

#

llm = ChatLiteLLM(model="groq/Llama3-70b-8192")
output_parser = StrOutputParser()
analysis_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
- Text history analysis reveals personality, behavior, preferences.
- Language use indicates literacy, education, regionality, style complexity, sentence structure.
- Communication skills apparent through message clarity, response length, topic adherence.
- Personality traits inferred from communication style, emoji use, expressiveness.
- Interests, hobbies discernible via discussion topics.
- Emotional state gauged from mood expressions, texting patterns.
- Social network, relationships inferred from contact interactions.
- Cultural background shown through references to customs, festivals.
- Professional, academic interests indicated by specific topic discussions.
- Values, beliefs reflected in topic responses, discussions.
- Behavioral patterns observed in communication timing, responsiveness.
""".strip(),
        ),
        (
            "ai",
            """
Consider the conversation so far:
{messages}

And the user's profile:
{user_profile}

Can you infer anything new about the user that isn't already in the profile?
Indicate your edits to the user profile (add/update/delete).
No narrative, just pure user profile revisions.
""".strip(),
        ),
    ]
)
analysis_chain = analysis_prompt | llm | output_parser

consolidate_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are a bullet list editor.  You only respond with edited bullet lists.
""".strip(),
        ),
        (
            "ai",
            """
Consider the original user profile:
{user_profile}

Here are suggested revisions:
{user_profile_revisions}

Generate a new, consolidated user profile based on these revisions.
No narrative, just a bullet list of user profile traits.
""".strip(),
        ),
    ]
)
user_profile_runnable = (
    {
        "user_profile_revisions": analysis_chain,
        "user_profile": itemgetter("user_profile"),
    }
    | consolidate_prompt
    | llm
    | output_parser
)

/Users/dothomps/.pyenv/versions/3.11.9/envs/lg/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/dothomps/.pyenv/versions/3.11.9/envs/lg/lib/python3.11/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_info" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
user_profile = """
(not provided)
""".strip()
res = user_profile_runnable.invoke(
    {
        "messages": [
            ("user", "Hi!"),
            ("ai", "Hello! How can I help you today?"),
            (
                "user",
                "I need help making a demo for a big tchnical presentation in 1 week!",
            ),
            ("ai", "I can help with that! What are you presenting on?"),
            (
                "user",
                "We are starting a new venture in AI using advanced agents as 'digital doubles' of professionals.",
            ),
            (
                "ai",
                "That sounds interesting! What kind of help do you need with the demo?",
            ),
            (
                "user",
                "I need to show how our agents can be used to automate tasks in a professional setting.",
            ),
            (
                "ai",
                "Got it! I can help you with that. What kind of tasks do you want to automate?",
            ),
        ],
        "user_profile": user_profile,
    }
)

print(res)

Here is the revised bullet list:

• **Interests:** AI, Advanced Agents, Automation, Professional Settings
• **Professional Background:** AI, Technology, or Entrepreneurship
• **Current Project:** Developing digital doubles of professionals
• **Communication Style:** Clear, concise, goal-oriented
• **Values:** Innovative, entrepreneurial, efficient
